# Iceberg - Cidades Brasileiras

Este notebook demonstra a criação, leitura e manipulação de uma tabela Iceberg usando o dataset `cidades_brasileiras.csv`.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

spark = SparkSession.builder     .appName("Iceberg - Cidades")     .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")     .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")     .config("spark.sql.catalog.spark_catalog.type", "hive")     .getOrCreate()

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("sigla", StringType(), True),
    StructField("ibge", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [ ]:
df = spark.read.csv("../data/cidades_brasileiras.csv", header=True, schema=schema)
df.show(5)

In [ ]:
df.writeTo("spark_catalog.default.cidades_iceberg").using("iceberg").createOrReplace()

In [ ]:
spark.sql("""
    INSERT INTO spark_catalog.default.cidades_iceberg VALUES
    (9999, 'Nova Esperança', 'Paraná', 'PR', 9999999, -23.5, -51.5)
""")

In [ ]:
spark.sql("UPDATE spark_catalog.default.cidades_iceberg SET cidade = 'Cidade Atualizada' WHERE id = 9999")

In [ ]:
spark.sql("DELETE FROM spark_catalog.default.cidades_iceberg WHERE id = 9999")

In [ ]:
df_final = spark.sql("SELECT * FROM spark_catalog.default.cidades_iceberg")
df_final.show(5)